### Recyclability Classifier, based on OpenAI's CLIP Model
Source: https://openai.com/research/clip

In [29]:
from common import *
from PIL import Image
import torch
import clip

# Initialise material classes
material_classes = list(material_class_mapping.values())

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, download_root='./clip')

image = preprocess(Image.open("dataset/waste_dataset/Paper/bandicam 2019-11-05 23-40-47-671.jpg")).unsqueeze(0).to(device)
text = torch.cat([clip.tokenize(f"a photo of an object made of {c}") for c in material_classes]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(len(material_classes))

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{material_classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

           Paper: 50.23%
         Plastic: 21.11%
           Metal: 15.82%
          Others: 10.53%
           Glass: 2.30%
